In [1]:
import spacy
import random
import pickle
from spacy import displacy
import pandas as pd

# import spacy

In [2]:
model = spacy.load ( 'en_core_web_sm' )
model.pipe_names

['tagger', 'parser', 'ner']

# import skill set

In [3]:
df = pd.read_excel('Correlation_Link.xls')
skill_Set = set( list( df['node1'] ) + list( df['node2'] ) )
skill_Set = list(skill_Set)
skill_Set = skill_Set + ['JSP','J2EE','UNIX','SYBASE','Windows','LINUX','Computer Science','NoSQL','SQL Server']
skill_Set = [w.lower() for w in skill_Set]
skill_Set

['languages',
 't-sql',
 'epidemiology',
 'word',
 'data',
 'creates',
 'mixpanel',
 'ai',
 'python/r',
 'automation',
 'spark',
 'marketing',
 'my sql',
 'microsoft excel',
 'operations research',
 'arcgis',
 'svn',
 'ssis',
 'rds',
 'social sciences',
 'architect',
 'dallas',
 'machine',
 'statistical data analytics',
 'kubernetes',
 'mdx',
 'service',
 'architecture',
 'mxnet',
 'stan',
 'statistical modelling',
 'power bi',
 'google cloud',
 'ms sql server',
 'business development',
 'audit',
 'redshift',
 'salesforce',
 'css',
 'gradient boosting',
 'databricks',
 'vsts',
 'tensorflow',
 'algorithms',
 'dashboards',
 'product development',
 'jquery',
 'java script',
 'statistical modeling',
 'github',
 'sales',
 'feature engineering',
 'c',
 'programming',
 'visual basic',
 'ms azure',
 'aws',
 'r studio',
 'the knowledge',
 'etl',
 'statistical analysis',
 'statistical genetics',
 'hive',
 'artificial intelligence',
 'mysql',
 'apache hadoop',
 'data analysis',
 'sqlite',
 'ph',


In [4]:
def is_tem_in_skill_set(ents, skill_Set):
    if str(ents).lower() in skill_Set:
        return True
    
    for tem in ents:
        if str(tem).lower() in skill_Set:
#             print( str(tem).lower() )
            return True
    
    return False

# import sentence contains DATE

In [5]:
with open('demo.pkl','rb') as out_data:
    data=pickle.load(out_data)

for sentence in data[:10]:
    print('-'*120)
    print(sentence)

------------------------------------------------------------------------------------------------------------------------
accurate status of projects aligned with the methodology being used (SCRUM/Iterative/ Waterfall)            Carry a Tier-2 support pager on a 6  8 week rotation           Support, maintenance, and development of JAVA web based and stand-alone applications
------------------------------------------------------------------------------------------------------------------------
A bachelors degree in Computer Science or Engineering and at least 6 years of Java experience working with server side applications, 7 years of experience can be substituted for a degree Candidates must be clearable US Citizens or hold at least an interim DOD security clearance
------------------------------------------------------------------------------------------------------------------------
- Minimum of 4-6 years working experience with Java/Unix/Oracle-based development.
-------------------

# test

In [6]:
sentence = data[ random.randint(0, len(data) ) ]

# sentence = 'Overall work experience:  1+ years with Java, J2EE application development.'
doc = model(sentence)
print( sentence )
print()

print( [chunks for chunks in doc.noun_chunks] )
print()

print( [chunks for chunks in doc.ents] )
print( [chunks for chunks in doc.ents if is_tem_in_skill_set(chunks,skill_Set)] )
print( [chunks for chunks in doc.ents if chunks.root.ent_type_ == 'DATE' and 'year' in chunks.root.text] )

This experience must include:   - Six years of experience in Business IT/C2 Support Systems.  

[This experience, - Six years, experience, Business IT/C2 Support Systems]

[Six years, Business IT/C2 Support Systems]
[Business IT/C2 Support Systems]
[Six years]


# take a sentence plot dependency tree

In [7]:
def get_Date_attribute( string ):
    print( string )
    doc = model( string )

    print()
    print( [chunk for chunk in doc.noun_chunks] )
    displacy.render(doc, style='dep', jupyter=True)
    
#     print( [chunks for chunks in doc.noun_chunks if chunks.root.text in skill_Set] )
    print( [chunks for chunks in doc.ents if is_tem_in_skill_set(chunks,skill_Set)] )
    print( [chunks for chunks in doc.ents if chunks.root.ent_type_ == 'DATE' and 'year' in chunks.root.text] )

In [8]:
# import random

sentence = data[ random.randint(0, len(data) ) ]
get_Date_attribute( sentence )

At least 5 years in Java development           

[At least 5 years, Java development]


[Java]
[At least 5 years]


# Link year attribute to entity

In [10]:
def is_father( word, root ):
    if word == root:
        return True
    
    while word != word.head:
        word = word.head
        if word == root:
            return True
    
    return False

In [11]:
# sentence = '2+ years working in an Agile environment'
# sentence ='years experience with Java development'
# sentence = 'This candidate needs to have 3-5 years of Java development experience, J2EE experience, Spring, and Hibernate experience.'
# sentence = '2 years experience developing service-oriented (SOA) architectures and EAI technologies such as Tibco'
# sentence = 'Java application development experience  Must have 3+ years'

# sentence = 'Minimum 5 years experience in working within a web development team'
# sentence = data[ random.randint(0, len(data) ) ]

sentence = '3+ years experience of Java application development'

# testing

In [12]:
for sentence in data:
    doc = model( sentence )
    
#     print( [ent for ent in doc.noun_chunks] )
    date_list = [chunks for chunks in doc.ents if chunks.root.ent_type_ == 'DATE' and 'year' in chunks.root.text]
    tem_list = [chunks for chunks in doc.ents if is_tem_in_skill_set(chunks,skill_Set)]

    if len(date_list) > 0 and len(tem_list):
        print( '-' * 120 )
        print( sentence )
        print()
        
        print( 'date_list:', date_list )
        print( 'tem_list:', tem_list)

        for date in date_list:
            Date = date
            for tem in tem_list:
                Flag = False

                if is_father( tem.root, Date.root ) or is_father( Date.root, tem.root ):
                    print( tem ,'-->' , Date )
                    Flag = True

                if Flag == False:
                    Date_Head = Date.root
                    while Flag == False and Date_Head != Date_Head.head:
                        Date_Head = Date_Head.head

                        if Date_Head.tag_ == 'NOUN' and is_father( tem.root, Date_Head ):
                            print( tem ,'-->' , Date )
                            Flag = True

                        if Date_Head.tag_ == 'NOUN':
                            break



                    #####
                    if Date_Head.dep_ == 'compound':
                        Date_Head = Date_Head.head

                        if is_father( tem.root, Date_Head ):
                            print( tem ,'-->' , Date )
                            Flag = True

                        elif Date_Head.dep_ == 'compound':
                            Date_Head = Date_Head.head

                            if is_father( tem.root, Date_Head ):
                                print( tem ,'-->' , Date )
                                Flag = True

------------------------------------------------------------------------------------------------------------------------
A bachelors degree in Computer Science or Engineering and at least 6 years of Java experience working with server side applications, 7 years of experience can be substituted for a degree Candidates must be clearable US Citizens or hold at least an interim DOD security clearance

date_list: [at least 6 years, 7 years]
tem_list: [Java]
Java --> at least 6 years
------------------------------------------------------------------------------------------------------------------------
- Minimum of 4-6 years working experience with Java/Unix/Oracle-based development.

date_list: [4-6 years]
tem_list: [Java/Unix/Oracle]
------------------------------------------------------------------------------------------------------------------------
development experience with JSF, Servlets, JSP, JSTL, and IceFaces, and Velocity templates -3 years professional experience in developing w

------------------------------------------------------------------------------------------------------------------------
This experience must include: - Six years of experience in Business IT/C2 Support Systems.  

date_list: [Six years]
tem_list: [Business IT/C2 Support Systems]
Business IT/C2 Support Systems --> Six years
------------------------------------------------------------------------------------------------------------------------
- Three years direct experience with UNIX, SYBASE, Windows and/or LINUX.

date_list: [Three years]
tem_list: [UNIX]
------------------------------------------------------------------------------------------------------------------------
- Experience on eCommerce & public customer facing web applications w/high transactional volume Excellent Java and SQL skills (4+ years)

date_list: [4+ years]
tem_list: [Excellent Java, SQL]
------------------------------------------------------------------------------------------------------------------------
B. 

------------------------------------------------------------------------------------------------------------------------
A bachelors degree in Computer Science or Engineering and at least 6 years of Java experience working with server side applications, 7 years of experience can be substituted for a degree Candidates must be clearable US Citizens or hold at least an interim DOD security clearance

date_list: [at least 6 years, 7 years]
tem_list: [Java]
Java --> at least 6 years
------------------------------------------------------------------------------------------------------------------------
•The successful candidate must possess a good understanding of enterprise architecture, web technologies, and have 3+ years of hands-on Java / J2EE development experience.

date_list: [3+ years]
tem_list: [Java / J2EE]
Java / J2EE --> 3+ years
------------------------------------------------------------------------------------------------------------------------
3-5 years academic or work expe

------------------------------------------------------------------------------------------------------------------------
A bachelors degree in Computer Science or Engineering and at least 6 years of Java experience working with server side applications, 7 years of experience can be substituted for a degree Candidates must be clearable US Citizens or hold at least an interim DOD security clearance

date_list: [at least 6 years, 7 years]
tem_list: [Java]
Java --> at least 6 years
------------------------------------------------------------------------------------------------------------------------
Minimum of 3 years experience in application development and system analysis  Knowledge and Skills Required:     Java, J2EE, JDBC, Struts, Spring, iBatis, Sybase and UDB Database     XML, XSLT,

date_list: [3 years]
tem_list: [Skills, J2EE, Sybase]
------------------------------------------------------------------------------------------------------------------------
 Minimum 5-7 years experie

------------------------------------------------------------------------------------------------------------------------
A bachelors degree in Computer Science or Engineering and at least 6 years of Java experience working with server side applications, 7 years of experience can be substituted for a degree Candidates must be clearable US Citizens or hold at least an interim DOD security clearance

date_list: [at least 6 years, 7 years]
tem_list: [Java]
Java --> at least 6 years
------------------------------------------------------------------------------------------------------------------------
The right individual will have 3+ years of experience in pure Java Development, J2EE, and Javascript.  

date_list: [3+ years]
tem_list: [Java Development, J2EE, Javascript]
Java Development --> 3+ years
J2EE --> 3+ years
Javascript --> 3+ years
------------------------------------------------------------------------------------------------------------------------
A bachelors degree in Compute

...

------------------------------------------------------------------------------------------------------------------------
A bachelors degree in Computer Science or Engineering and at least 6 years of Java experience working with server side applications, 7 years of experience can be substituted for a degree Candidates must be clearable US Citizens or hold at least an interim DOD security clearance

date_list: [at least 6 years, 7 years]
tem_list: [Java]
Java --> at least 6 years
------------------------------------------------------------------------------------------------------------------------
years of related work experienceExperience of at least 2-3 Java/J2EE projects

date_list: [years]
tem_list: [Java/J2EE]
Java/J2EE --> years
------------------------------------------------------------------------------------------------------------------------
Allstate Insurance Company is looking for a Java Developer with 2 - 3 years Java programming experience.  

date_list: [2 - 3 years]
te

------------------------------------------------------------------------------------------------------------------------
The environment is Java, JavaScript, XML, Eclipse, TestLink, Oracle, Linux, Windows 7     Required Skills:    - Development experience with Java, JavaScript and XML    - 3+ years of software configuration management experience supporting multi-project development    - 3+ years of experience managing multi-tier application development    - 3+ years of experience with build, source control, and configuration management tools such as Ant, Subversion, ClearCase, Jira, Hudson,

date_list: [3+ years]
tem_list: [Java, JavaScript, XML, Eclipse, Oracle, Linux, Skills, Java, JavaScript, XML, Jira]
Java --> 3+ years
------------------------------------------------------------------------------------------------------------------------
A bachelors degree in Computer Science or Engineering and at least 6 years of Java experience working with server side applications, 7 years of e

------------------------------------------------------------------------------------------------------------------------
A bachelors degree in Computer Science or Engineering and at least 6 years of Java experience working with server side applications, 7 years of experience can be substituted for a degree Candidates must be clearable US Citizens or hold at least an interim DOD security clearance

date_list: [at least 6 years, 7 years]
tem_list: [Java]
Java --> at least 6 years
------------------------------------------------------------------------------------------------------------------------
A bachelors degree in Computer Science or Engineering and at least 6 years of Java experience working with server side applications, 7 years of experience can be substituted for a degree Candidates must be clearable US Citizens or hold at least an interim DOD security clearance

date_list: [at least 6 years, 7 years]
tem_list: [Java]
Java --> at least 6 years
----------------------------------

------------------------------------------------------------------------------------------------------------------------
Job Specifications: - A bachelors degree in Computer Science or Engineering and at least 6 years of Java experience working with server side applications,

date_list: [at least 6 years]
tem_list: [Java]
Java --> at least 6 years
------------------------------------------------------------------------------------------------------------------------
We have won the Columbus Business First "Fast 50" award 4 times in the last 5 years because of our strong growth which translates into more and better opportunities for our consulting staff members.   

date_list: [the last 5 years]
tem_list: [the Columbus Business First]
------------------------------------------------------------------------------------------------------------------------
The ideal candidate will have 5-10 years of strong background developing complex web applications using Java/J2EE technologies as well 

------------------------------------------------------------------------------------------------------------------------
Ideal Java Developer candidates will have 3+ years of J2EE experience in a Web Environment.  

date_list: [3+ years]
tem_list: [J2EE]
J2EE --> 3+ years
------------------------------------------------------------------------------------------------------------------------
Contract     TEKsystems has partnered with a local government integrator to support their need for a Java developer with 3-5 years of experience working in a J2EE environment.

date_list: [3-5 years]
tem_list: [Java, J2EE]
J2EE --> 3-5 years
------------------------------------------------------------------------------------------------------------------------
Java Developer - 4+ years of Java experience   Location: Irving/

date_list: [years]
tem_list: [Java Developer - 4, Java]
Java Developer - 4 --> years
Java --> years
-----------------------------------------------------------------------------

In [69]:
def Extract_Info_from_JD( sentence ):
    skill_list = []
    skill_attribute_list = []
    
    doc = model( sentence )
    
    date_list = [chunks for chunks in doc.ents if chunks.root.ent_type_ == 'DATE' and 'year' in chunks.root.text]
    tem_list  = [chunks for chunks in doc.ents if is_tem_in_skill_set(chunks,skill_Set)]
    
    skill_list = tem_list
    if len(date_list) > 0 and len(tem_list):
        for date in date_list:
            Date = date
            for tem in tem_list:
                Flag = False

                if is_father( tem.root, Date.root ) or is_father( Date.root, tem.root ):
#                     print( '****',tem, '-->', Date )
                    skill_attribute_list.append( str( str(tem) + '-->' + Date.text ) )
                    
                    Flag = True

                if Flag == False:
                    Date_Head = Date.root
                    while Flag == False and Date_Head != Date_Head.head:
                        Date_Head = Date_Head.head

                        if Date_Head.tag_ == 'NOUN' and is_father( tem.root, Date_Head ):
#                             print( '****',tem, '-->', Date )
                            skill_attribute_list.append( str( str(tem) + '-->' + Date.text ) )
                            
                            Flag = True

                        if Date_Head.tag_ == 'NOUN' and Date_Head.dep_ != 'compound':
                            break

                    #####
                    if Date_Head.dep_ == 'compound':
                        Date_Head = Date_Head.head

                        if is_father( tem.root, Date_Head ):
#                             print( '****',tem, '-->', Date )
                            skill_attribute_list.append( str( str(tem) + '-->' + Date.text ) )
                            
                            Flag = True

                        elif Date_Head.dep_ == 'compound':
                            Date_Head = Date_Head.head

                            if is_father( tem.root, Date_Head ):
#                                 print( '****',tem, '-->', Date )
                                skill_attribute_list.append( str( str(tem) +'-->' + Date.text ) )
                                
                                Flag = True
                                
    return skill_list, skill_attribute_list

# import Job Description

In [55]:
import pandas as pd

nlp =  spacy.load('en')

data = pd.read_csv(r'E:/dataset/Top30.csv',usecols = ['Query','Description'])
data = data[data['Query'].isin([ 'Java Developer'])]
data.head(5)

,Query,Description
6,Java Developer,<P><STRONG>As a member of the Web and Portal D...
7,Java Developer,<BR>\r<TABLE border=0 cellSpacing=0 cellPaddin...
11,Java Developer,<strong>Application Developer-Senior-Java<br>\...
118,Java Developer,<b>Responsibilities:</b> Kforce is seeking a m...
132,Java Developer,<b>Responsibilities:</b> Our client is looking...


In [56]:
from bs4 import BeautifulSoup

def clean_text(text):
        text = text.replace(r'\\n', ' ').replace(r'\\r', ' ').replace(r'\\t', ' ').replace(r'\r', ' ').replace(r'\n', ' ').replace(r'\t', ' ').replace('\xa0','')

        soup = BeautifulSoup(text,'html.parser')
        return soup.get_text()

In [57]:
data['Description'] = data['Description'].apply(clean_text)
JD_set = list( data['Description'] )

# Test: get a random JD output

In [129]:
import random

text = JD_set[ random.randint(0, len(JD_set) ) ]
text = nlp( text )

total_skill_list = []
total_skill_attribute_list = []
for sent in text.sents:
    print( sent )
    skill_list ,skill_attribute_list = Extract_Info_from_JD( str(sent) )
    
    total_skill_list += skill_list
    total_skill_attribute_list += skill_attribute_list

print()
print()
print( '=' * 120 )
print( 'Extract Result:' )

print( 'Skill list:',  list(set(total_skill_list)) )
for tem in total_skill_attribute_list:
    print( tem )

We are looking to identify at least 5 more developers for an Army Contract in Springfield, VA.  
Many of the requirements below are ideal
but if you don't possess all of them training will be provided on the job.  
Must be able to obtain a DOD Secret Clearance.     
Position Description  Designs, develops, and implements web-based Java applications to support business requirements.
Follows approved life cycle methodologies, creates design documents, and performs program coding and testing.
Resolves technical issues through troubleshooting, debugging, research, and investigation.  
Familiar with software development concepts, practices, and procedures (i.e. Java, EJB 3.0, JSP, JSF, HTML, SQL, Javascript).
Relies on extensive experience and judgment to plan and accomplish goals.   
Principle Duties and Responsibilities:  Experience in building enterprise software or large scale web application required Strong Java and J EE software development required Experience in servlets and JSP deve